In [24]:
!pip install git+https://github.com/SheffieldSolar/PV_Live-API

  Cloning https://github.com/SheffieldSolar/PV_Live-API to /tmp/pip-req-build-ia6t6ko3
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime, date

In [26]:
df = pd.read_csv('data/consumption2017-P2.csv')
consumption_df = df[list(map(lambda x: len(x)>4,df.POSTCODE))]

In [27]:
all_postcodes = []
for i in range(100):
    consumption_postcodes = list(consumption_df.iloc[100*i:100*i+100]['POSTCODE'])
    assert consumption_postcodes is not None
    payload = dict(postcodes=consumption_postcodes)
    
    assert payload is not None
    result = requests.post('https://api.postcodes.io/postcodes', data=payload).json()
    assert result['status'] is 200
    results = result['result']
    postcodes = []
    for result in results:
        if result['result'] is None:
            print(result)
            continue
        postcodes.append({'POSTCODE':result['result']['postcode'],'Longitude':result['result']['longitude'],'Latitude':result['result']['latitude']})
    
    all_postcodes.append(list(postcodes))

{'query': 'LA6 3GQ', 'result': None}
{'query': 'LA9 4', 'result': None}
{'query': 'LE1 1LF', 'result': None}
{'query': 'LE1 5NE', 'result': None}
{'query': 'LE1 7GP', 'result': None}
{'query': 'LE12 8ZZ', 'result': None}
{'query': 'LE12 B5X', 'result': None}
{'query': 'LE15 0OO', 'result': None}
{'query': 'LE15 7AL', 'result': None}
{'query': 'LE15 7E', 'result': None}
{'query': 'LE17 4HA', 'result': None}
{'query': 'LE2 0OO', 'result': None}
{'query': 'LE2 1EN', 'result': None}
{'query': 'LE3 1QS', 'result': None}
{'query': 'LE3 3LT', 'result': None}


In [28]:
all_postcodes = sum(all_postcodes,[])

In [29]:
postcodes_df = pd.DataFrame(all_postcodes)

In [30]:
postcodes_df = df.set_index('POSTCODE').join(postcodes_df.set_index('POSTCODE')).dropna()

In [31]:
long_to_X = (np.array(postcodes_df.Longitude)+8)//(10/90)
lat_to_Y = (np.array(postcodes_df.Latitude)-49)//(10/72)
tiles_data = list(zip(long_to_X,lat_to_Y))
tiles_df = pd.DataFrame(tiles_data, columns=['X','Y'], index=postcodes_df.index, dtype='int64')


In [32]:
x_min = 990
y_min = 660

In [33]:
tiles_df = postcodes_df.join(tiles_df)

In [34]:
tiles_df = tiles_df.dropna()
tiles_df = tiles_df.reset_index()

In [35]:
#tiles_df.to_json('data/consumption-P2.json')

In [36]:
max(tiles_df.Y)

39

In [48]:
tiles_df = pd.read_json('data/consumption-P1.json')

In [49]:
tiles_df.tail()

,POSTCODE,Number of meters,Consumption (kWh),Mean consumption (kWh),Median consumption (kWh),Latitude,Longitude,X,Y
99995,BN3 1EB,13,37534,2887,2280,50.825389,-0.158406,35,6
99996,BN3 1ED,39,102356,2625,2238,50.825306,-0.159361,35,6
99997,BN3 1EE,54,203331,3765,2633,50.824638,-0.159827,35,6
99998,BN3 1EF,54,132793,2459,2288,50.823996,-0.160136,35,6
99999,BN3 1EG,39,127016,3257,2142,50.823718,-0.158287,35,6
